In [106]:
# Import NGSolve
from ngsolve import *
from ngsolve.webgui import Draw 

# Ecuaciones de FitzHugh-Nagumo

Buscamos resolver las ecuaciones de FitzHugh-Nagumo en un dominio $\Omega_{T}=\Omega \times (0,T)$ con $\Omega \subset \mathbb{R}^{d}$ y frontera $\Sigma_{T}=\Sigma \times (0,T)$:

\begin{align*}
	\left\{
	\begin{array}{ll}
		\dfrac{\partial \phi}{\partial t}-D\Delta \phi -c_{1}\phi (\phi-\alpha)(1-\phi)-c_{2}r=I_{\text{app}}(x,t), \quad (x,t) \in \Omega_{T}\\
		\\
		\dfrac{\partial r}{\partial t}-b(\phi-dr)=0,  \quad (x,t) \in \Omega_{T}\\
        \\
        D\nabla \phi\cdot \mathbf{n}=0, \quad    (x,t) \in \Sigma_{T},\\ 
        \\
	   \phi(x,0)=\phi_{0}(x),\\
       \\
	   r(x,0)=r_0(x).
	\end{array}
	\right. 
\end{align*}

In [107]:
# Datos del Problema
D = 0.5
alpha = 0.4
c1 = 100
c2 = 0.03
b = 0.15
d = 0.9
Iapp = 0

# Formulación Variacional

La formulación variacional de las ecuaciones de FitzHugh-Nagumo están dadas por:

\begin{equation*}\label{Eq2}
	\left\{
	\begin{array}{ll}
		\displaystyle\int_{\Omega}	\dfrac{\partial \phi}{\partial t}\varphi \, + D\displaystyle\int_{\Omega} \nabla \phi \cdot \nabla \varphi  =	\displaystyle\int_{\Omega} (c_{1}\phi (\phi-\alpha)(1-\phi)-c_{2}r)\varphi+\displaystyle\int_{\Omega} I_{\text{app}}(x,t)\varphi, \\
		\\
			\displaystyle\int_{\Omega}	\dfrac{\partial r}{\partial t}\psi  =	\displaystyle\int_{\Omega}	(b\phi-bdr)\psi,
	\end{array}
	\right.
\end{equation*}
en donde $\varphi, \psi \in H_{0}^{1}(\Omega)$.

# Integración Temporal: Forward-Euler (tiempo)

Usaremos Forward - Euler para aproximar la solución, en donde se asume conocido $t=t_{n}$. Para esto, consideraremos la semi-discretización temporal en la formulación variacional con paso de tiempo $\Delta t$:

\begin{equation*}
	\left\{
	\begin{array}{ll}
		\displaystyle\int_{\Omega}	 \frac{\phi_h^{n+1}-\phi_h^{n} }{\Delta t}\varphi \, + D\displaystyle\int_{\Omega} \nabla \phi_h^{n} \cdot \nabla \varphi  -	\displaystyle\int_{\Omega} (c_{1}\phi_h^{n} (\phi_h^{n}-\alpha)(1-\phi_h^{n})-c_{2}r_h^{n})\varphi =	\displaystyle\int_{\Omega}	I_{\text{app}}(x,t_{n})\varphi,\\
        \\
    \displaystyle\int_{\Omega}	\frac{r_h^{n+1}-r_h^{n} }{\Delta t}\psi -  	\displaystyle\int_{\Omega}	(b\phi_h^{n}-bdr_h^{n})\psi=0.
	\end{array}
	\right.
\end{equation*}

Para esto, se definen los funcionales:
\begin{align*}
  a((\phi, r),(\varphi, \psi)) &= D \int_{\Omega} \nabla \phi \cdot \nabla \varphi\, d\Omega - \int_{\Omega}(c_{1}\phi (\phi-\alpha)(1-\phi)-c_{2}r)\varphi-\int_{\Omega} (b\phi-bdr)\psi \, d\Omega,\\
   m((\phi, r),(\varphi, \psi)) &= \int_{\Omega}  \phi \varphi \, d\Omega + \int_{\Omega} r \psi \, d\Omega,\\
   b(\varphi, \psi)) &= \int_{\Omega} I_{\text{app}} \varphi \, d\Omega.
\end{align*}

De esta manera, podemos reescribir la semi-discretización temporal en virtud de los funcionales $a,m,b$. En efecto:

\begin{align*}
 \frac{m((\phi_{h}^{n+1}-\phi_{h}^{n}, r_{h}^{n+1}-r_{h}^{n}),(\varphi, \psi))}{\Delta t}+a((\phi_{h}^{n},r_{h}^{n}),(\varphi, \psi)) &= b(\varphi,\psi).
\end{align*}

Consideremos $\delta \phi_{h}^{n+1} =\phi_{h}^{n+1}-\phi_{h}^{n}$ y $\delta r_{h}^{n+1} =r_{h}^{n+1}-r_{h}^{n}$. Multiplicando por $\Delta t$, se obtiene:

\begin{align*}
 m((\delta \phi_{h}^{n+1}, \delta r_{h}^{n+1}),(\varphi, \psi)) &= \Delta t [b(\varphi,\psi)-a((\phi_{h}^{n},r_{h}^{n}),(\varphi, \psi))].
\end{align*}

Para utilizar Métodos de Elementos Finitos (FEM), discretizaremos el espacio $\Omega$ con una malla $\mathcal{T}_{h}$ de $\Sigma$. Para esto, cosndieraremos el espacio de polinomios lineales a trozos $V_{N}$ sobre los elementos de $\mathcal{T}_{h}$. De esta manera, a cada funcional $a,b,m$ se le asocia una matriz de ensamblaje. Por consiguiente, en cada iteración se debe resolver el sistema:


\begin{align*}
M \delta U_{h}^{n+1} &= \Delta t [b(\varphi,\psi)-A(U_{h}^{n})].
\end{align*}

Tenemos $\delta U_{h}^{n+1} = (\delta  \phi_{h}^{n+1}, \delta r_{h}^{n+1})$ y $ U_{h}^{n} = (\phi_{h}^{n}, \delta r_{h}^{n})$.

In [108]:
# Mallado y Geometría
from netgen.geom2d import unit_square
geo = unit_square
# Tamaño del Mallado
hraw = 0.1
mesh = Mesh(geo.GenerateMesh(maxh=hraw))
# Preguntar si debo incluir condición de frontera

In [109]:
# Time Loop
import time
# Tiempo final
tend = 1.0
# Paso de tiempo
CFL = 0.5
dt = CFL * hraw**2
tn = 0

In [110]:
# FEM
order = 2
V = H1(mesh, order=order)
W = H1(mesh, order=order)
fes = V*W
(phi,r), (varphi,psi) = fes.TnT() 

# Formas Bilineales
a = BilinearForm(fes)
a += (D*grad(phi)*grad(varphi))*dx 
a += -c1*phi*(phi-alpha)*(1-phi)*varphi*dx
a += -(b*phi-b*d*r)*psi*dx
a

m = BilinearForm(fes)
m += phi*varphi*dx 
m += r*psi*dx
m

b = LinearForm(fes)
b += Iapp*varphi*dx
b += 0*psi*dx
b

Tomaremos $I_{\text{app}}=0$.

In [111]:
# Matriz de Ensamblaje
a.Assemble()
m.Assemble()
b.Assemble()

mstar = m.mat.CreateMatrix()
mstar.AsVector().data =  m.mat.AsVector() + dt * a.mat.AsVector()

# ¿Cómo definir mstar.Asvector() con la forma lineal b asociada al impulso externo I_app? No estoy seguro si m.star.AsVector()
# es correcto. Preguntar.

invmstar = mstar.Inverse(freedofs=fes.FreeDofs())

In [112]:
# Condiciones Iniciales
t = Parameter(0)
phi0 = 1 + 0.5*cos(4*pi*x)*cos(4*pi*y)
r0 = 1 + 0.5*cos(8*pi*x)*cos(8*pi*y)
gfu = GridFunction(fes)
gfu.components[0].Set(phi0) 
gfu.components[1].Set(r0)

In [113]:
# Solución
scene = Draw(gfu.components[0], mesh)
res = gfu.vec.CreateVector()
while tn < tend - 0.5 * dt:
    res.data = -dt * a.mat * gfu.vec
    tn += dt
    t.Set(tn)
    res.data += dt * b.vec  
    gfu.vec.data += invmstar * res
    
    print("\r t = ",tn,end="")
    scene.Redraw()
    time.sleep(0.1)
print("")

WebGuiWidget(value={'gui_settings': {}, 'ngsolve_version': '6.2.2204', 'mesh_dim': 2, 'order2d': 2, 'order3d':…

 t =  1.000000000000000776
